## Running Quicscript QFT using Qiskit
Reference: https://quantumcomputinguk.org/tutorials/quantum-fourier-transform-in-qiskit

### Loading Library

In [2]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT
from qiskit_aer import Aer
import numpy as np

pi = np.pi

### Loading QFT

In [5]:
q = QuantumRegister(5,'q')
c = ClassicalRegister(5,'c')

circuit = QuantumCircuit(q,c)

circuit.x(q[4])
circuit.x(q[2])
circuit.x(q[0])
qft = QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=False, insert_barriers=False, name='qft')
circuit.append(qft, qargs=[0, 1, 2, 3, 4])
qfti = QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=True, insert_barriers=False, name='qfti')
circuit.append(qfti, qargs=[0, 1, 2, 3, 4])
circuit.measure(q,c)
circuit.draw(output='mpl')
print(circuit)

     ┌───┐┌──────┐┌───────┐┌─┐            
q_0: ┤ X ├┤0     ├┤0      ├┤M├────────────
     └───┘│      ││       │└╥┘┌─┐         
q_1: ─────┤1     ├┤1      ├─╫─┤M├─────────
     ┌───┐│      ││       │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 qft ├┤2 qfti ├─╫──╫─┤M├──────
     └───┘│      ││       │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├┤3      ├─╫──╫──╫─┤M├───
     ┌───┐│      ││       │ ║  ║  ║ └╥┘┌─┐
q_4: ┤ X ├┤4     ├┤4      ├─╫──╫──╫──╫─┤M├
     └───┘└──────┘└───────┘ ║  ║  ║  ║ └╥┘
c: 5/═══════════════════════╩══╩══╩══╩══╩═
                            0  1  2  3  4 


## Running Qiskit Simulator

In [18]:
from qiskit_aer import Aer
simulator = Aer.get_backend('statevector_simulator')
result = simulator.run(circuit.decompose(reps=6)).result()
sv = result.get_statevector(circuit)
arr = []
for i, x in enumerate(sv.data):
    data = x
    if data != 0:
        arr.append((i,data))
for x in arr:
    print("{0:b} => {1}".format(x[0],x))

10101 => (21, np.complex128(1-7.088954791644251e-16j))


> `10101` is the only data, hence the probability of it's 100%

## Translate Qiskit to QuICScript

In [15]:
## Translate to Quicscript
from qiskit_quicscript_translator import get_quic_circuit_file_string
file_string = get_quic_circuit_file_string(circuit.decompose(reps=6))

In [19]:
file_string

'# 5\n{U 3.141592653589793 0.0 3.141592653589793}\nUIIII,\n{U 3.141592653589793 0.0 3.141592653589793}\nIIUII,\n{U 3.141592653589793 0.0 3.141592653589793}\nIIIIU,\n{U 1.5707963267948966 0.0 3.141592653589793}\nIIIIU,\n{U 0.0 0.0 0.7853981633974483}\nIIIIU,\nIIINC,\n{U 0.0 0.0 -0.7853981633974483}\nIIIUI,\nIIINC,\n{U 0.0 0.0 0.7853981633974483}\nIIIUI,\n{U 1.5707963267948966 0.0 3.141592653589793}\nIIIUI,\n{U 0.0 0.0 0.7853981633974483}\nIIIUI,\n{U 0.0 0.0 0.39269908169872414}\nIIIIU,\nIINIC,\n{U 0.0 0.0 -0.39269908169872414}\nIIUII,\nIINIC,\n{U 0.0 0.0 0.39269908169872414}\nIIUII,\nIINCI,\n{U 0.0 0.0 -0.7853981633974483}\nIIUII,\nIINCI,\n{U 0.0 0.0 0.7853981633974483}\nIIUII,\n{U 1.5707963267948966 0.0 3.141592653589793}\nIIUII,\n{U 0.0 0.0 0.7853981633974483}\nIIUII,\n{U 0.0 0.0 0.39269908169872414}\nIIIUI,\n{U 0.0 0.0 0.19634954084936207}\nIIIIU,\nINIIC,\n{U 0.0 0.0 -0.19634954084936207}\nIUIII,\nINIIC,\n{U 0.0 0.0 0.19634954084936207}\nIUIII,\nINICI,\n{U 0.0 0.0 -0.3926990816987241

In [22]:
# Save to file
f = open("quicscript_qft.txt", "w")
f.write(file_string)
f.close()

### Result from running on QuICScript File-Based
```
Result of File execution:
00000,0.00%;
00001,0.00%;
00010,0.00%;
00011,0.00%;
00100,0.00%;
00101,0.00%;
00110,0.00%;
00111,0.00%;
01000,0.00%;
01001,0.00%;
01010,0.00%;
01011,0.00%;
01100,0.00%;
01101,0.00%;
01110,0.00%;
01111,0.00%;
10000,0.00%;
10001,0.00%;
10010,0.00%;
10011,0.00%;
10100,0.00%;
10101,100.00%;
10110,0.00%;
10111,0.00%;
11000,0.00%;
11001,0.00%;
11010,0.00%;
11011,0.00%;
11100,0.00%;
11101,0.00%;
11110,0.00%;
11111,0.00%;
```

> Evidently see that `10101` is 100% which is the same as the qiskit simulator as it's the only result from the simulation